In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/iws_state", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/iws_event", True)

#display(dbutils.fs.ls("/FileStore/"))

Out[1]: False

In [0]:
!pip install pm4py
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
import pandas as pd
from pyspark.sql.functions import col, struct, explode, collect_list,lit,array,split, expr
from pyspark.sql.types import StructType, StructField, StringType, ArrayType,FloatType,DoubleType,IntegerType,TimestampType
from time import sleep

     |████████████████████████████████| 1.9 MB 4.4 MB/s 
     |████                            | 1.7 MB 8.5 MB/s eta

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 5.0 MB 5.7 MB/s 
     |████████████████████████████████| 47 kB 3.2 MB/s 
     |████████████████████████████████| 78 kB 4.9 MB/s 
     |████████████████████████████████| 1.6 MB 9.4 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26117 sha256=c13bff3d97c3a0df9f68977eb036e6724bd25d5f9afc22e5210799182cf6bcad
  Stored in directory: /root/.cache/pip/wheels/ab/fa/1b/75d9a713279796785711bd0bad8334aaace560c0bd28830c8c
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24575 sha256=2eb75458bf0580a0e0f560967777338860adf1564109cf87bec03f489c5c86a6
  Stored in directory: /root/.cache/pip/wheels/89/e5/de/6966007cf223872eedfbebbe0e074534e72e9128c8fd4b55eb
Successfully built intervaltree pydotplus
You should consid

In [0]:
event_log = pm4py.read_xes("/tmp/BPI_2012_Sim_2k_random_0_95.xes")
incoming_events = log_converter.apply(event_log, variant=log_converter.Variants.TO_DATA_FRAME)
schema = StructType([
    StructField("resource", StringType(), True),
    StructField("event", StringType(), True),
    StructField("transition", StringType(), True),
    StructField("time_stamp", TimestampType(), True),
    StructField("trace_id", StringType(), True),
])
incoming_events = spark.createDataFrame(incoming_events,schema=schema)
incoming_events = incoming_events.select("event","time_stamp","trace_id")


parsing log, completed traces ::   0%|          | 0/2000 [00:00<?, ?it/s]

In [0]:
class AlphabetService:
    def __init__(self):
        self.activity_to_alphabet = {}
        self.alphabet_to_activity = {}
        self.char_counter = 64  

    def alphabetize(self, label):
        if label not in self.activity_to_alphabet:
            self.char_counter += 1
            self.activity_to_alphabet[label] = chr(self.char_counter)
            self.alphabet_to_activity[chr(self.char_counter)] = label
            #print("label",label,"char",chr(self.char_counter))
        return self.activity_to_alphabet[label]

    def clear(self):
        self.activity_to_alphabet.clear()
        self.alphabet_to_activity.clear()
        self.char_counter = 64


def process_trace(trace, nodes,depth, current_node='root'):
    last_node = current_node #root
    previous_n_nodes = ['root']

    nodes['root'] = {'label': 'root', 'parent': None, 'level':0, 'direct_children': set(),'direct_children_labels':set(),'nth_children':set()}
    for event in trace:
        event_name = event['alphabetized_label']
        node_id = f"{last_node}-{event_name}"
        if node_id not in nodes:
            nodes[node_id] = {'label': event_name, 'parent': last_node if last_node != 'root' else 'root', 'level':int((len(node_id)-4)/2), 'direct_children': set(),'direct_children_labels':set(),'nth_children':set()}
        
        #update the children of the parent node
        if len(previous_n_nodes)>depth:
            previous_n_nodes = previous_n_nodes[1:]
        for node in previous_n_nodes:
            start = len(node)
            end = len(node_id)-1
            events_between = tuple(node_id[start+1:end-1].split("-"))
            sub_array = (node_id,event_name,int((len(node_id)-4)/2),events_between)
            nodes[node]['nth_children'].add(sub_array)
        if last_node != 'root':
            nodes[last_node]['direct_children'].add(node_id)
            nodes[last_node]['direct_children_labels'].add(event_name)
        previous_n_nodes.append(node_id)
        last_node = node_id

    return nodes

def convert_sets_to_lists(obj):
    if isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, dict):
        return {k: convert_sets_to_lists(v) for k, v in obj.items()}
    else:
        return obj

event_log = pm4py.read_xes("/tmp/BPI_2012_1k_sample.xes")
dataframe = log_converter.apply(event_log, variant=log_converter.Variants.TO_DATA_FRAME)
labels_trace = dataframe[["concept:name", "case:concept:name"]]
grouped_traces = labels_trace.groupby("case:concept:name",sort=False)

alphabet_service = AlphabetService()
nodes = {}
for trace, group in grouped_traces:
    group['alphabetized_label'] = group["concept:name"].apply(alphabet_service.alphabetize)
    nodes = process_trace(group.to_dict('records'), nodes,3)

labels = alphabet_service.activity_to_alphabet
schema = StructType([
    StructField("event", StringType(), True),
    StructField("label", StringType(), True)
])
df_labels = spark.createDataFrame(list(labels.items()), schema=schema)

data = [{"node_id": node_id, **convert_sets_to_lists(node_data)} for node_id, node_data in nodes.items()]
#df_nodes = spark.createDataFrame(data)

schema = StructType([
    StructField("node_id", StringType(), True),
    StructField("label", StringType(), True),
    StructField("level", IntegerType(), True),
    StructField("direct_children", ArrayType(StringType()), True),
    StructField("direct_children_labels", ArrayType(StringType()), True),
    StructField("nth_children", ArrayType(
        StructType([
            StructField("node_id", StringType(), True),
            StructField("label", StringType(), True),
            StructField("level", IntegerType(), True),
            StructField("events_between", ArrayType(StringType()), True)
        ])
    ), True),
    StructField("parent", StringType(), True)
])
df_nodes = spark.createDataFrame(data, schema=schema)


parsing log, completed traces ::   0%|          | 0/1000 [00:00<?, ?it/s]

In [0]:
df_nodes.createOrReplaceTempView("iws_model")
df_labels.createOrReplaceTempView("iws_labels")


In [0]:
#%sql
#CREATE OR REPLACE TABLE iws_model AS
#SELECT "root" AS node_id, "-" AS label, array("A") AS children_labels, array("root-A") AS children_id, 
#    array(
#        struct("root-A" AS node_id, "A" AS label, 1 as level, array() as events_between),
#        struct("root-A-B" AS node_id, "B" AS label, 2 as level, array("A") as events_between),
#        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array("A","B") as events_between), 
#        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array("A","B") as events_between)
#    ) AS nth_children
#UNION ALL 
#SELECT "root-A", "A", array("B"), array("root-A-B"), 
#    array(
#        struct("root-A-B" AS node_id, "B" AS label, 2 as level, array() as events_between),
#        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array("B") as events_between),
#        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array("B") as events_between),
#        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array("B","D") as events_between)
#    )
#UNION ALL 
#SELECT "root-A-B", "B", array("X", "D"), array("root-A-B-X", "root-A-B-D"), 
#    array(
#        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array() as events_between), -- "A-B-C" now "A-B-X"
#        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array() as events_between),
#        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array("D") as events_between)
#    )
#UNION ALL 
#SELECT "root-A-B-X", "X", array(), array(),
#    array()
#UNION ALL 
#SELECT "root-A-B-D", "D", array("C"), array("root-A-B-D-C"), 
#    array(
#        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array() as events_between),
#        struct("root-A-B-D-C-D" AS node_id, "D" AS label, 5 as level, array("C") as events_between),
#        struct("root-A-B-D-C-C" AS node_id, "C" AS label, 5 as level, array("C") as events_between)
#    )
#UNION ALL 
#SELECT "root-A-B-D-C", "C", array("D", "C"), array("root-A-B-D-C-D", "root-A-B-D-C-C"), 
#    array(
#        struct("root-A-B-D-C-D" AS node_id, "D" AS label, 5 as level, array() as events_between),
#        struct("root-A-B-D-C-C" AS node_id, "C" AS label, 5 as level, array() as events_between)
#    )
#UNION ALL 
#SELECT "root-A-B-D-C-D", "D", array(), array(), array()
#UNION ALL 
#SELECT "root-A-B-D-C-C", "C", array(), array(), array();
#
#CREATE OR REPLACE TABLE iws_event 
#SELECT "A" event, CURRENT_TIMESTAMP() time_stamp, "trace_id_0" trace_id;
#
#CREATE OR REPLACE TABLE iws_labels 
#SELECT "A" AS event, "A"  AS label
#UNION ALL
#SELECT "B","B" 
#UNION ALL
#SELECT "C","C" 
#UNION ALL
#SELECT "D","D" 
#UNION ALL
#SELECT "X","X" ;
#
#CREATE OR REPLACE TABLE iws_state
#(trace_id STRING, ts TIMESTAMP, current_node STRING,current_id STRING,cost_of_alignment INTEGER,previous_events STRING, trace STRING, execution_sequence STRING,event_level INTEGER,#current_event_level INTEGER,current_node_level INTEGER,write_ts TIMESTAMP);
#--event level to filter out the latest alignments later


In [0]:
%sql
CREATE OR REPLACE TABLE iws_event (event STRING, time_stamp TIMESTAMP, trace_id STRING);

CREATE OR REPLACE TABLE iws_state
(trace_id STRING, ts TIMESTAMP, current_node STRING,current_id STRING,cost_of_alignment INTEGER,previous_events STRING, trace STRING, execution_sequence STRING,event_level INTEGER,current_event_level INTEGER,current_node_level INTEGER);
--event level to filter out the latest alignments later

In [0]:
import random 

activities = ["A","B","C","D","X"]
trace_id_lower = 0
trace_id_upper = 9

def insert_event(event=None, trace_id=None):
    if not event:
        event = random.choice(activities)
    if not trace_id:
        trace_id = 0 #random.randint(trace_id_lower, trace_id_upper)
    spark.sql(f"INSERT INTO iws_event SELECT '{event}', CURRENT_TIMESTAMP(), 'trace_id_{trace_id}'")

In [0]:
import pyspark.sql.functions as F 
from time import sleep
event_df = spark.readStream.table("iws_event").withWatermark("time_stamp", "30 minutes")

In [0]:
event_df.createOrReplaceTempView("events")

In [0]:

%sql
CREATE OR REPLACE TEMP VIEW latest_state AS 
SELECT DISTINCT trace_id, ts, current_node, current_id, cost_of_alignment,previous_events,trace,execution_sequence,event_level, max_event_level as current_event_level,current_node_level,rn,write_ts FROM (
SELECT *, row_number() OVER (PARTITION BY trace_id,current_id order by event_level desc,cost_of_alignment asc, len(previous_events) desc) rn,
max(event_level) OVER (PARTITION BY trace_id) AS max_event_level FROM iws_state
) WHERE rn = 1 --and event_level > current_event_level -2
--need to only get the values where when the node_id is the same, then the one with the highest event level is saved
-- Trace needs to be like this since we are possibly using past events (which are not updated) in the future

In [0]:
#if looking at older events, cost should be difference in event levels also number of skips >> between is same number
#TODO FIX IF EVENT IS FIRST COMING IN (fix with adding root row automatically)
#current level -> used to track what event the state is from if the state is used in the future
#event level -> used to track the current event level (how many events there have been)
#current_node_level -> used to track what level the node is, used in calculation
spark.sql("""
SELECT trace_id,ts,exploded.*,previous_events,CURRENT_TIMESTAMP() as write_ts FROM(
SELECT trace_id,ts,concat(model_moves,exploded_struct) as sub_exploded,previous_events FROM(
SELECT 
      e.trace_id, 
      e.time_stamp as ts,
      concat(e.previous_events,e.event) as previous_events,
      transform(
        filter(m.nth_children, x -> x.label LIKE e.event),
        x -> struct(e.event as current_node, x.node_id as current_id, e.cost_of_alignment + abs(x.level - current_node_level-1) + current_event_level - event_level as cost_of_alignment,  concat(coalesce(trace,"root"),repeat(">>",abs(x.level - current_node_level-1)),e.event) as trace,CONCAT(execution_sequence, coalesce(CONCAT_WS('', x.events_between)),e.event) as execution_sequence, current_event_level+1 as event_level, current_event_level + 1 as current_event_level,x.level as current_node_level)
      ) as model_moves,
      array(struct(e.current_node as current_node, m.node_id as current_id, e.cost_of_alignment + e.current_event_level - e.event_level + 1 as cost_of_alignment, concat(trace, e.event) as trace, concat(execution_sequence, ">>")as execution_sequence, e.current_event_level + 1 as event_level, current_event_level + 1 as current_event_level, current_node_level))
      AS exploded_struct
    FROM 
      (SELECT e.trace_id,e.time_stamp,l.label as event, COALESCE(r.current_node, "-") as current_node, COALESCE(r.current_id, "root") as current_id, coalesce(cost_of_alignment, 0) as cost_of_alignment, coalesce(r.previous_events,"") as previous_events , coalesce(trace,"") as trace, coalesce(execution_sequence,"") as execution_sequence, coalesce(event_level, 0) as event_level, coalesce(current_event_level, 0) as current_event_level,coalesce(current_node_level,0) as current_node_level FROM events e LEFT JOIN latest_state r ON e.trace_id = r.trace_id JOIN iws_labels l on e.event = l.event) e 
    JOIN iws_model m ON e.current_id = m.node_id ))
  LATERAL VIEW explode(sub_exploded) t AS exploded
""").writeStream.format("delta").outputMode("append").option("checkpointLocation","/tmp/delta/state_append_51/").toTable("iws_state")

Out[78]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fc23e74ebe0>

In [0]:
def simulate_stream_write(dataframe, delay=5):
    sorted_df = dataframe.orderBy("time_stamp")
    for row in sorted_df.collect():
        spark.sql(f"INSERT INTO iws_event SELECT '{row.event}', CURRENT_TIMESTAMP(), '{row.trace_id}'")
        sleep(delay)

from threading import Thread

# Start the simulation in a background thread to keep the main thread responsive
thread = Thread(target=simulate_stream_write, args=(incoming_events, 5))
thread.start()

In [0]:
%sql
SELECT * FROM latest_state where trace_id = "trace_id_1"

trace_id,ts,current_node,current_id,cost_of_alignment,previous_events,trace,execution_sequence,event_level,current_event_level,current_node_level,rn,write_ts
trace_id_1,2024-04-22T15:49:31.395+0000,-,root,3,ABX,ABX,>>>>>>,3,3,0,1,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,A,root-A,2,ABX,ABX,A>>>>,3,3,1,1,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,B,root-A-B,1,ABX,ABX,AB>>,3,3,2,1,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,C,root-A-B-D-C,3,ACX,A>>>>CX,ABDC>>,3,3,4,1,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,X,root-A-B-X,0,ABX,ABX,ABX,3,3,3,1,2024-04-22T15:49:45.462+0000


In [0]:
%sql
SELECT * FROM iws_state where trace_id = "trace_id_1"
-- vaata mis state on

trace_id,ts,current_node,current_id,cost_of_alignment,previous_events,trace,execution_sequence,event_level,current_event_level,current_node_level,write_ts
trace_id_1,2024-04-22T15:49:31.395+0000,C,root-A-B-D-C,3,ACX,A>>>>CX,ABDC>>,3,3,4,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,X,root-A-B-X,0,ABX,ABX,ABX,3,3,3,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,B,root-A-B,1,ABX,ABX,AB>>,3,3,2,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,X,root-A-B-X,2,ABX,AB>>X,A>>BX,3,3,3,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,A,root-A,2,ABX,ABX,A>>>>,3,3,1,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,X,root-A-B-X,4,ABX,AB>>>>X,>>>>ABX,3,3,3,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:31.395+0000,-,root,3,ABX,ABX,>>>>>>,3,3,0,2024-04-22T15:49:45.462+0000
trace_id_1,2024-04-22T15:49:20.220+0000,B,root-A-B,0,AB,AB,AB,2,2,2,2024-04-22T15:49:30.508+0000
trace_id_1,2024-04-22T15:49:20.220+0000,A,root-A,1,AB,AB,A>>,2,2,1,2024-04-22T15:49:30.508+0000
trace_id_1,2024-04-22T15:49:20.220+0000,B,root-A-B,2,AB,A>>B,>>AB,2,2,2,2024-04-22T15:49:30.508+0000


In [0]:
%sql
SELECT * from events e JOIN iws_labels l on e.event = l.event

event,time_stamp,trace_id,event,label
D,2024-04-21T15:53:10.957+0000,trace_id_7,D,D
D,2024-04-21T16:25:12.285+0000,trace_id_9,D,D
D,2024-04-21T15:54:33.475+0000,trace_id_8,D,D
D,2024-04-21T15:54:03.542+0000,trace_id_8,D,D
D,2024-04-21T14:59:13.727+0000,trace_id_4,D,D
D,2024-04-21T14:59:08.781+0000,trace_id_6,D,D
D,2024-04-21T14:58:56.406+0000,trace_id_5,D,D
D,2024-04-21T14:58:11.746+0000,trace_id_4,D,D
D,2024-04-21T14:37:51.967+0000,trace_id_0,D,D
D,2024-04-21T14:36:34.630+0000,trace_id_0,D,D


In [0]:
#ACBXC
insert_event("A","1")
sleep(1)
insert_event("C","1")
sleep(1)
insert_event("B","1")
sleep(1)
insert_event("X","1")
#sleep(2)
#insert_event("C","2")
#sleep(6)
#insert_event("D","2")
#sleep(6)
#insert_event("C","2")
#sleep(6)
#insert_event("D","2")


#for i in range(7):
#    insert_event(trace_id = "1")
#    sleep(6)
    #insert_event(trace_id = "5")
    #insert_event(trace_id = "6")
#    
# lisa uus event. Vaata uuesti mis state on (ülemine cell)
# võid katsetada lisada suvalist eventi vahepeal, nt X